In [1]:
import numpy as np
import cv2
import json
import torch
import os
import random
from mmdet.apis import (inference_detector, init_detector, show_result_pyplot)

In [2]:

# config = '/home/oem/mmdetection/work_dirs/dconv_c3-c5_cascade_mask_rcnn_x101_32x4d/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py'
# checkpoint = '/home/oem/mmdetection/work_dirs/dconv_c3-c5_cascade_mask_rcnn_x101_32x4d/v3_final.pth'
# config = '/home/oem/mmdetection/work_dirs/binary_cascade/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py'
# checkpoint = '/home/oem/mmdetection/work_dirs/binary_cascade/binary_3set.pth'
config = '/home/oem/mmdetection/work_dirs/nocluster/nocluster.py'
checkpoint = '/home/oem/mmdetection/work_dirs/nocluster/nocluster_set2.pth'
model = init_detector(config, checkpoint, device = 'cuda:0')
json_path = 'test.json'
CLASSES = ('big_house', 'house', 'container', 'big_building', 'road', 'building',
            'house_cluster', 'building_cluster', 'etc', 'empty_soil', 'empty_paved',
            'empty_etc')


In [3]:
def pointInRectRatio(point,rect, ratio = 1):    
    x1, y1, x2, y2 = rect    
    x, y = point
    w_offset  = ((x2 - x1) * ratio) / 2
    h_offset = ((y2 - y1) * ratio) / 2
    if ((x1 + w_offset) < x and x < (x2 - w_offset)):
        if ((y1 + h_offset) < y and y < (y2 - h_offset)):
            return True
    return False

def pointInRect(point,rect):    
    x1, y1, x2, y2 = rect    
    x, y = point
    
    if (x1 < x and x < x2):
        if (y1 < y and y < y2):
            return True
    return False

In [11]:
for i in range(10):    
    img14 = '/home/oem/data/alpha/images/4_{}_14.jpg'.format(i)
    img18 = '/home/oem/data/alpha/images/4_{}_18.jpg'.format(i)        
    # img14 = '/home/oem/data/alpha/gamma/4_{}_14.jpg'.format(i)
    # img18 = '/home/oem/data/alpha/gamma/4_{}_18.jpg'.format(i)    
    result14 = inference_detector(model, img14)
    result18 = inference_detector(model, img18)
    thresh = 0.4
    detect_ratio = 1
    test14_bbox = result14[0][:9]
    test18_bbox = result18[0][:9]
    cnt = 0
    diff_bbox = []    
    for class_14, class_18 in zip(test14_bbox, test18_bbox):
        if cnt != 4:
            past_results = [item[:4] for item in class_14 if item[4] > thresh]
            now_results = [item[:4] for item in class_18 if item[4] > thresh]        
            for now in now_results:
                now_center = [(now[0] + now[2]) / 2, (now[1] + now[3]) / 2]
                if not any(pointInRect(now_center, past) for past in past_results): 
                    diff_bbox.append(now)
        cnt += 1
    result_18 = cv2.imread(img18)
    result_14 = cv2.imread(img14)
    thickness = 3    
    for box in diff_bbox:      
        start = (box[0], box[1])  
        end =  (box[2], box[3])          
        color = random.sample(range(50, 255), 3)
        result_18 = cv2.rectangle(result_18, start, end, color, thickness)
        result_14 = cv2.rectangle(result_14, start, end, color, thickness)
    inference_14 = show_result_pyplot(model, img14, result14)
    inference_18 = show_result_pyplot(model, img18, result18)
    row1 = np.hstack([inference_18, result_18])
    row2 = np.hstack([inference_14, result_14])
    final_result = np.vstack([row1, row2])
    # final_result = np.vstack([result_18, result_14])
    cv2.imwrite('result_{}.jpg'.format(i), final_result)  



In [30]:


# print(diff_bbox)

True

In [16]:
# mask = {"size": [995, 1442], "counts": "fWiP12in00[QO5`n0;I5L6K3K4M3N2N2O1L5K4N2N2N2M3M3N2O1N2O1O2N1O1N2N2O100O100O100O1O101O0O10001O000001O01O01O000000010O001000O000100O1O00100000000O00100O0010000O100O1O2O0O1O011N10O01O1O001O001O1O1O1O001O1O00001O0O2O1N2O0O2N2O1M2O2L3N3M2O2N2M2M4L4M3M4L3M5FlZ[7"}
# mask_new, contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
#                                                         cv2.CHAIN_APPROX_SIMPLE)

AttributeError: 'dict' object has no attribute 'astype'